In [15]:
# Importamos librerías necesarias
import telebot

import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import sqlite3

from datetime import datetime, timedelta
import dateparser

import nltk
from nltk.corpus import stopwords

In [16]:
# Variables de configuración
TOKEN = '7738846991:AAFEmojbVE6rP3jtPCVfu_EdSeRNJkdnW-0'
CSV_FILE = '..\data\processed\Articulos_LLM5.csv'  # ← tu archivo CSV

# Cargamos el CSV de artículos
articulos_df = pd.read_csv(CSV_FILE)
articulos_df = articulos_df.drop_duplicates()

# Vamos a unir titulo + contenido + fechas + precios + edad
articulos_df['texto_completo'] = (
    articulos_df['titulo'].fillna('') + '. ' +
    articulos_df['contenido'].fillna('') + ' ' +
    articulos_df['fechas_contexto'].fillna('') + ' ' +
    articulos_df['Precios'].fillna('') + ' '
)

In [17]:
conn = sqlite3.connect('conversaciones.db')
cursor = conn.cursor()

# Crear tabla
cursor.execute('''
    CREATE TABLE IF NOT EXISTS mensajes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        username TEXT,
        nombre TEXT,
        mensaje TEXT,
        respuesta TEXT,
        fecha TEXT
    )
''')

conn.commit()
conn.close()

In [18]:
def guardar_conversacion(user_id, username, nombre, mensaje, respuesta):
    conn = sqlite3.connect('conversaciones.db')
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO mensajes (user_id, username, nombre, mensaje, respuesta, fecha)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (
        user_id,
        username,
        nombre,
        mensaje,
        respuesta,
        datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    ))
    conn.commit()
    conn.close()

In [19]:
# Asegúrate de tener descargado el paquete de stopwords de NLTK
nltk.download('stopwords')

# Cargamos las stopwords en español
stopwords_es = stopwords.words('spanish')

# Creamos el TF-IDF usando las stopwords en español
tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords_es)

# Ahora sí, transformamos
tfidf_matrix = tfidf_vectorizer.fit_transform(articulos_df['texto_completo'])


[nltk_data] Downloading package stopwords to C:\Users\Abdon.RAMIREZ-
[nltk_data]     BRICEN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
# Inicializamos el bot
bot = telebot.TeleBot(TOKEN)

# --- Nueva función: interpretar fechas en la pregunta ---

def interpretar_fecha_usuario(pregunta):
    pregunta = pregunta.lower()
    hoy = datetime.now()

    if 'hoy' in pregunta:
        return hoy.date(), hoy.date()
    
    elif 'mañana' in pregunta:
        return (hoy + timedelta(days=1)).date(), (hoy + timedelta(days=1)).date()
    
    elif 'semana que viene' in pregunta or 'semana próxima' in pregunta or 'semana siguiente' in pregunta:
        inicio = hoy + timedelta(days=(7 - hoy.weekday()))  # lunes próximo
        fin = inicio + timedelta(days=6)  # domingo de la próxima semana
        return inicio.date(), fin.date()
    
    elif 'esta semana' in pregunta:
        # Desde hoy hasta domingo
        inicio = hoy
        fin = hoy + timedelta(days=(6 - hoy.weekday()))
        return inicio.date(), fin.date()

    elif 'fin de semana' in pregunta or 'sábado' in pregunta or 'domingo' in pregunta:
        # Calcula el próximo sábado y domingo
        dias_hasta_sabado = (5 - hoy.weekday()) % 7
        sabado = hoy + timedelta(days=dias_hasta_sabado)
        domingo = sabado + timedelta(days=1)
        return sabado.date(), domingo.date()
    
    else:
        # No se detectó ninguna fecha relevante
        return None, None

def detectar_ciudad(pregunta):
    ciudades = ["madrid", "barcelona", "malaga", "sevilla", "valencia", "zaragoza"]
    pregunta = pregunta.lower()
    for ciudad in ciudades:
        if ciudad in pregunta:
            return ciudad.capitalize()
    return None

# --- Nueva función: convertir fechas del dataset ---
def preparar_fechas_dataset(df):
    fecha_inicio_parseada = []
    fecha_fin_parseada = []

    for inicio, fin in zip(df['fecha_inicio'].fillna(''), df['fecha_fin'].fillna('')):
        # Parsear fecha inicio
        try:
            if inicio:
                fecha_inicio_dt = dateparser.parse(str(inicio), languages=['es'])
                fecha_inicio_parseada.append(fecha_inicio_dt.date() if fecha_inicio_dt else None)
            else:
                fecha_inicio_parseada.append(None)
        except Exception as e:
            fecha_inicio_parseada.append(None)

        # Parsear fecha fin
        try:
            if fin:
                fecha_fin_dt = dateparser.parse(str(fin), languages=['es'])
                fecha_fin_parseada.append(fecha_fin_dt.date() if fecha_fin_dt else None)
            else:
                fecha_fin_parseada.append(None)
        except Exception as e:
            fecha_fin_parseada.append(None)

    df['fecha_inicio_parseada'] = fecha_inicio_parseada
    df['fecha_fin_parseada'] = fecha_fin_parseada

    return df# Procesamos el dataframe para agregar la columna de fechas
articulos_df = preparar_fechas_dataset(articulos_df)

def es_saludo_o_gratitud(pregunta):
    frase = pregunta.lower().strip()
    saludos = ['hola', 'buenas', 'buenos días', 'buenas tardes', 'buenas noches']
    agradecimientos = ['gracias', 'muchas gracias', 'te agradezco', 'mil gracias']

    for palabra in saludos + agradecimientos:
        if palabra in frase:
            return palabra
    return None

# --- Función para buscar la mejor respuesta ---
def buscar_respuesta(pregunta_usuario):
    intencion = es_saludo_o_gratitud(pregunta_usuario)
    if intencion:
        if intencion in ['gracias', 'muchas gracias', 'te agradezco', 'mil gracias']:
            return "🙏 ¡Gracias a ti por la consulta! Te esperamos pronto. 😊"
        else:
            return "👋 ¡Hola! ¿En qué ciudad te gustaría encontrar actividades? Puedo ayudarte con Madrid, Barcelona, Málaga, etc."

    fecha_inicio_consulta, fecha_fin_consulta = interpretar_fecha_usuario(pregunta_usuario)
    ciudad_consulta = detectar_ciudad(pregunta_usuario)

    if not ciudad_consulta and not fecha_inicio_consulta:
        return (
            "🤔 No tengo información para lo que me indicas. Intenta preguntarme algo como:\n\n"
            "📍 ¿Qué actividades hay en Madrid hoy?\n"
            "📅 ¿Qué puedo hacer este fin de semana en Barcelona?\n"
            "🎯 ¿Hay algo gratuito en Sevilla esta semana?"
        )

    # DEBUG
    print(f"DEBUG 🛠 Fecha: {fecha_inicio_consulta} → {fecha_fin_consulta}")
    print(f"DEBUG 🛠 Ciudad: {ciudad_consulta}")

    filtrados = articulos_df.copy()

    if fecha_inicio_consulta and fecha_fin_consulta:
        filtrados = filtrados[
            (filtrados['fecha_inicio_parseada'].notna()) &
            (filtrados['fecha_fin_parseada'].notna()) &
            (filtrados['fecha_inicio_parseada'] <= fecha_fin_consulta) &
            (filtrados['fecha_fin_parseada'] >= fecha_inicio_consulta)
        ]

    if ciudad_consulta:
        filtrados = filtrados[filtrados['texto_completo'].str.lower().str.contains(ciudad_consulta.lower())]

    if filtrados.empty:
        return "🤔 No encontré actividades para esa ciudad y fecha. ¡Intenta preguntar otra cosa!"

    # TF-IDF
    tfidf_matrix_filtrada = tfidf_vectorizer.transform(filtrados['texto_completo'])
    pregunta_vectorizada = tfidf_vectorizer.transform([pregunta_usuario])
    similitudes = cosine_similarity(pregunta_vectorizada, tfidf_matrix_filtrada)

    mejores_indices = np.argsort(similitudes[0])[::-1]
    urls_vistas = set()
    respuesta = ""
    count = 0

    for idx in mejores_indices:
        articulo = filtrados.iloc[idx]
        if articulo['url'] in urls_vistas:
            continue
        urls_vistas.add(articulo['url'])

        respuesta += (
            f"✅ *{articulo['titulo']}*\n"
            f"{articulo['contenido'][:300]}...\n"
            f"💰 Precio: {articulo['Precios']}\n"
            f"🔗 [Más info]({articulo['url']})\n\n"
        )
        count += 1
        if count == 3:
            break

    return respuesta if respuesta else "🤔 No encontré actividades relacionadas, intenta otra consulta."

# --- Manejadores del bot ---
@bot.message_handler(commands=['start'])
def start(message):
    chat_id = message.chat.id

    # Enviamos la imagen
    with open('..\QHCN.jpg', 'rb') as photo:
        bot.send_photo(chat_id, photo)

    # Enviamos el mensaje de bienvenida
    mensaje_bienvenida = (
        "👋 ¡Hola! Soy tu asistente virtual para ayudarte a buscar actividades.\n\n"
        "Puedes preguntarme cosas como:\n"
        "📌 ¿Qué actividad puedo hacer hoy?\n"
        "📌 A mi hijo/a le gusta el deporte, ¿qué podría hacer esta semana?\n"
        "📌 ¿Qué actividades hay en Sevilla hoy?\n"
        "📌 ¿Qué actividades puedo hacer este fin de semana gratis?\n\n"
        "¡Estoy aquí para ayudarte! 🎯"
    )

    bot.send_message(chat_id, mensaje_bienvenida, parse_mode='Markdown')

@bot.message_handler(func=lambda message: True)
def responder(message):
    consulta = message.text
    respuesta = buscar_respuesta(consulta)

    guardar_conversacion(
        user_id=message.from_user.id,
        username=message.from_user.username,
        nombre=message.from_user.first_name,
        mensaje=consulta,
        respuesta=respuesta
    )

    bot.send_message(message.chat.id, respuesta, parse_mode='Markdown')

# Lanzamos el bot
print("🤖 Bot activo y esperando mensajes...")
bot.polling()

🤖 Bot activo y esperando mensajes...
DEBUG 🛠 Fecha: 2025-05-08 → 2025-05-08
DEBUG 🛠 Ciudad: Barcelona


In [23]:
conn = sqlite3.connect('../data/BBDD_conversaciones/conversaciones.db')
df = pd.read_sql_query("SELECT * FROM mensajes", conn)
conn.close()

df

,id,user_id,username,nombre,mensaje,respuesta,fecha
0,1,1532965212,None,.,Qué actividades tienes para hacer este fin de ...,✅ *FUNBOX en Madrid: saltos y obstáculos en el...,2025-05-07 12:41:32
1,2,1532965212,None,.,Y en Barcelona?,✅ *10 sitios imprescindibles para visitar en B...,2025-05-07 12:41:44
2,3,1532965212,None,.,"Semana santa ya ha pasado, quiero para este fi...",✅ *¡Viaja al pasado! Descubre el Valle de los ...,2025-05-07 12:42:05
3,4,1532965212,None,.,Quiero este fin de semana en Barcelona,✅ *‘Magic Days’: un pueblo navideño en pleno c...,2025-05-07 12:42:20
4,5,1532965212,None,.,Gracias,✅ *Un lago de conciertos: las Artes y las Cien...,2025-05-07 12:42:33
5,6,1532965212,None,.,Hola,👋 ¡Hola! ¿En qué ciudad te gustaría encontrar ...,2025-05-07 12:55:19
6,7,1532965212,None,.,Quiero información sobre actividades en Barcel...,✅ *‘Magic Days’: un pueblo navideño en pleno c...,2025-05-07 12:58:03
